## Notebook to create the atmospheric forcing files for ROMS input using Tim's set-up


In this notebook:
- The period from 1993 to 2018 is selected



From Tim's matlab files:
Prepare the ERA5 analysis data to be used in ROMS
- Change units
- Compute relative humidity from temperature and dewpoint temperature
- Multiply by scale factor and add offset (it seems that when importing variables from a dataset the offset and scale factor are automatically applied)
- Loop on years


Written in Tim's paper:
The atmospheric boundary conditions were derived from ERA5 (Copernicus Climate Change Service
(C3S), 2017). ERA5 is based on the IFS41r2 Earth system model and is available for 1979 to a few months
before present. The atmospheric forcing was applied using bulk formulae based on (Fairall et al., 1996).
We prescribed solar radiation daily using an idealized diurnal cycle and precipitation 12‐hourly. Winds at
10 m above sea level, atmospheric pressure, cloud coverage, air temperature, and relative air humidity at
2 m above sea level were prescribed on a 6‐hourly basis.






Dewi said the netcdf files contain seconds

In [1]:
# Import necessary packages

import dask
import numpy as np
import xarray as xr

from tqdm import tqdm

dask.config.set({"array.slicing.split_large_chunks": False})

In [2]:
# Import data

era5_hourly = xr.open_mfdataset('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/hourly/*.nc')
era5_6hourly = xr.open_mfdataset('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/*.nc')


### Create atmospheric forcing for 6hourly data


The reference datum for ERA5 data is 01-01-1970 whereas roms uses 01-01-1948 as a reference

In [5]:
era5_6hourly = era5_6hourly.sortby('latitude') # Make sure the latitude values are ascending for both the dimension and all variables

In [6]:
# Figure out the reference time for datetime timesteps
np.datetime64(0, 'D')

numpy.datetime64('1970-01-01')

In [7]:
# Obtain seconds between '1948-01-01T00:00:00' and '1970-01-01T00:00:00'
sec_shift = np.datetime64('1948-01-01T00:00:00').astype(int)*-1 # Unit: seconds

In [8]:
# Create dimensions
lons = era5_6hourly.longitude.values
lats = era5_6hourly.latitude.values
time_1970 = era5_6hourly.time.values.astype(int)/10**9     # Create a time array of seconds since 01-01-1970
time_1948 = time_1970 + sec_shift                          # Create a time array of seconds since 01-01-1948

In [9]:
# Create variables
msl = era5_6hourly.msl.values * 0.01   # Convert from Pa to mbar
t2m = era5_6hourly.t2m.values - 273.15 # Convert from Kelvin to Celsius
tcc = era5_6hourly.tcc.values
u10 = era5_6hourly.u10.values
v10 = era5_6hourly.v10.values

In [10]:
# Compute Relative Humidity Qair from t2m and d2m using eq. 12 of DOI:10.1175/BAMS-86-2-225
L = 2.5*10**6.    # Enthalpy of vaporization at 273.15K (J/kg)
Rw = 461.5        # Gaz constant for water vapor (J/K/kg)
t2m_K = era5_6hourly.t2m.values
d2m_K = era5_6hourly.d2m.values
Qair = 100*np.exp(-L*(t2m_K - d2m_K)/(Rw*t2m_K*d2m_K))

In [11]:
# Create DataSet

# Define variables
data_vars = {'Uwind':(['time', 'lat', 'lon'], u10, 
                      {'units': 'meter second-1', 'long_name': 'surface u-wind component', 'standard_name' : 'surface_eastward_wind', 'time' : 'time', 'coordinat' : 'lon lat'}),
             'Vwind':(['time', 'lat', 'lon'], v10, 
                      {'units': 'meter second-1', 'long_name': 'surface v-wind component', 'standard_name' : 'surface_northward_wind', 'time' : 'time', 'coordinat' : 'lon lat'}),
             'cloud':(['time', 'lat', 'lon'], tcc, 
                      {'units': 'nondimensional', 'long_name': 'cloud fraction', 'standard_name' : 'cloud_area_fraction', 'time' : 'time', 'coordinat' : 'lon lat'}),
             'Pair':(['time', 'lat', 'lon'], msl, 
                     {'units': 'millibar ', 'long_name': 'surface air pressure', 'standard_name' : 'surface_air_pressure', 'time' : 'time', 'coordinat' : 'lon lat'}),
             'Tair':(['time', 'lat', 'lon'], t2m, 
                     {'units': 'Celsius', 'long_name': 'surface air temperature', 'standard_name' : 'surface_air_temperature', 'time' : 'time', 'coordinat' : 'lon lat'}),
             'Qair':(['time', 'lat', 'lon'], Qair, 
                     {'units': 'percentage', 'long_name': 'surface air relative humidity', 'standard_name' : 'surface_relative_humidity', 'time' : 'time', 'coordinat' : 'lon lat'}),
            }

# Define coordinates
coords = {'time':(['time'], time_1948, {'units': 'seconds since 1948-01-01 00:00:0.0', 'long_name': 'time', 'standard_name': 'time', 'calendar': 'gregorian', }),
         'lon':(['lon'], lons, {'units': '°E', 'long_name': 'Longitude', 'standard_name': 'Longitude'}),
         'lat':(['lat'], lats, {'units': '°N', 'long_name': 'Latitude', 'standard_name': 'Latitude'})}



# Create Dataset
ERA5_an = xr.Dataset(data_vars = data_vars,
                    coords = coords,
                    )



In [14]:
ERA5_an.time

<xarray.DataArray 'time' (time: 105192)>
array([6.315840e+07, 6.318000e+07, 6.320160e+07, ..., 2.335241e+09,
       2.335262e+09, 2.335284e+09])
Coordinates:
  * time     (time) float64 6.316e+07 6.318e+07 6.32e+07 ... 2.335e+09 2.335e+09
Attributes:
    units:          seconds since 1948-01-01 00:00:0.0
    long_name:      time
    standard_name:  time
    calendar:       gregorian

In [15]:
# Save Dataset
ERA5_an.to_netcdf('/Users/iriskeizer/Documents/ROMS/timnorthsea8/forcing/ERA5_NorthAtlantic_an_ForROMS.nc')

### Create atmospheric forcing for 12hourly and daily data

In [33]:
# Shift every hour forward (such that the data is given at each hour as the sum over the next hour)
era5_hourly_shft = era5_hourly.shift(time = -1)

# Select period 1993 - 2018
era5_hourly_shft = era5_hourly_shft.where((era5_hourly_shft.time.dt.year > 1992) & (era5_hourly_shft.time.dt.year < 2019), drop = True)

# Sort latitudes ascending
era5_hourly_shft = era5_hourly_shft.sortby('latitude')

# Group by 12h or by day and obtain sum
era5_12hourly = era5_hourly_shft.resample(time = '12H').sum('time')
era5_daily = era5_hourly_shft.resample(time='1D').sum('time')



In [34]:
# Figure out the reference time for datetime timesteps
np.datetime64(0, 'D')

numpy.datetime64('1970-01-01')

In [35]:
# Obtain seconds between '1948-01-01T00:00:00' and '1970-01-01T00:00:00'
sec_shift = np.datetime64('1948-01-01T00:00:00').astype(int)*-1

In [49]:
# Create dimensions
lons = era5_12hourly.longitude.values
lats = era5_12hourly.latitude.values
time_12h_1970 = era5_12hourly.time.values.astype(int)/10**9                                               # Create a time array of seconds since 01-01-1970 (ns->s)
time_d_1970 = era5_daily.time.values.astype(int)/10**9                                                    # Create a time array of seconds since 01-01-1970 (ns->s)

time_12h_1948 = time_12h_1970 + sec_shift                       # Create a time array of seconds since 01-01-1948
time_d_1948 = time_d_1970 + sec_shift

#time_12h_1948 = time_12h_1948 - 6*60*60 # Remove 6 hours to center the flux to the middle of the forcasting time (12h) instead of the beginning
#time_d_1948 = time_d_1948               # Add 12 hours to have midday values

In [37]:
# Obtain fluxes

tp = era5_12hourly.tp.values * 1000/(12*60*60) # From m to kg/m**2/s
ssr = era5_daily.ssr.values /(24*60*60)

In [54]:
# Create DataSet

# Define variables
data_vars = {'rain':(['time_rain', 'lat', 'lon'], tp, 
                     {'units': 'kilogram meter-2 second-1', 'long_name': 'rain fall rate', 'standard_name' : 'precipitation_flux', 'time' : 'time_rain', 'coordinates' : 'lon lat'}),
            'swrad':(['time_swrad', 'lat', 'lon'], ssr, 
                    {'units': 'watt meter-2', 'long_name': 'solar shortwave radiation flux', 'standard_name' : 'net_downward_shortwave_flux_at_sea_water_surface', 'time' : 'time_swrad', 'coordinates' : 'lon lat'})}

# Define coordinates
coords = {'time_rain':(['time_rain'], time_12h_1948, {'units': 'seconds since 1948-01-01 00:00:0.0', 'long_name': 'time', 'standard_name': 'time', 'calendar': 'gregorian', }),
          'time_swrad':(['time_swrad'], time_d_1948, {'units': 'seconds since 1948-01-01 00:00:0.0', 'long_name': 'time', 'standard_name': 'time', 'calendar': 'gregorian', }),
         'lon':(['lon'], lons, {'units': '°E', 'long_name': 'Longitude', 'standard_name': 'Longitude'}),
         'lat':(['lat'], lats, {'units': '°N', 'long_name': 'Latitude', 'standard_name': 'Latitude'})}

# Create Dataset
ERA5_fc = xr.Dataset(data_vars = data_vars,
                    coords = coords,
                    )




In [52]:
ERA5_fc

<xarray.Dataset>
Dimensions:     (time_rain: 18992, lat: 137, lon: 153, time_swrad: 9496)
Coordinates:
  * time_rain   (time_rain) float64 1.42e+09 1.42e+09 ... 2.241e+09 2.241e+09
  * time_swrad  (time_swrad) float64 1.42e+09 1.42e+09 ... 2.24e+09 2.241e+09
  * lon         (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.5 13.75 14.0
  * lat         (lat) float32 32.0 32.25 32.5 32.75 ... 65.25 65.5 65.75 66.0
Data variables:
    rain        (time_rain, lat, lon) float32 2.822e-05 2.016e-05 ... 0.0004858
    swrad       (time_swrad, lat, lon) float64 111.8 112.1 ... 0.03286 0.0348

In [55]:
# Save Dataset
ERA5_fc.to_netcdf('/Users/iriskeizer/Documents/ROMS/timnorthsea8/forcing/ERA5_NorthAtlantic_fc_ForROMS.nc', engine = 'netcdf4')

### Compare my file and Dewi's file

In [3]:
data_an = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/timnorthsea8/forcing/ERA5_NorthAtlantic_an_ForROMS.nc')
data_fc = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/timnorthsea8/forcing/ERA5_NorthAtlantic_fc_ForROMS.nc')

In [4]:
data_dewi_an = xr.open_dataset('/Users/iriskeizer/Downloads/ERAint_an_1993_NA_ForROMS.nc')
data_dewi_fc = xr.open_dataset('/Users/iriskeizer/Downloads/ERAint_fc_1993_NA_ForROMS.nc')

In [4]:
data_an

<xarray.Dataset>
Dimensions:  (time: 37984, lat: 137, lon: 153)
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Data variables:
    Uwind    (time, lat, lon) float32 ...
    Vwind    (time, lat, lon) float32 ...
    cloud    (time, lat, lon) float32 ...
    Pair     (time, lat, lon) float32 ...
    Tair     (time, lat, lon) float32 ...
    Qair     (time, lat, lon) float32 ...

In [18]:
data_an.Uwind

<xarray.DataArray 'Uwind' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          meter second-1
    long_name:      surface u-wind component
    standard_name:  surface_eastward_wind
    time:           time
    coordinat:      lon lat

In [19]:
data_an.Vwind

<xarray.DataArray 'Vwind' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          meter second-1
    long_name:      surface v-wind component
    standard_name:  surface_northward_wind
    time:           time
    coordinat:      lon lat

In [12]:
data_an.cloud

<xarray.DataArray 'cloud' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          nondimensional
    long_name:      cloud fraction
    standard_name:  cloud_area_fraction
    time:           time
    coordinat:      lon lat

In [13]:
data_an.Pair

<xarray.DataArray 'Pair' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          millibar 
    long_name:      surface air pressure
    standard_name:  surface_air_pressure
    time:           time
    coordinat:      lon lat

In [14]:
data_an.Tair

<xarray.DataArray 'Tair' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          Celsius
    long_name:      surface air temperature
    standard_name:  surface_air_temperature
    time:           time
    coordinat:      lon lat

In [15]:
data_an.Qair

<xarray.DataArray 'Qair' (time: 37984, lat: 137, lon: 153)>
[796182624 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 ... 2018-12-31T18:00:00
  * lon      (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat      (lat) float32 32.0 32.25 32.5 32.75 33.0 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          percentage
    long_name:      surface air relative humidity
    standard_name:  surface_relative_humidity
    time:           time
    coordinat:      lon lat

In [5]:
data_fc

<xarray.Dataset>
Dimensions:     (time_rain: 18992, lat: 137, lon: 153, time_swrad: 9496)
Coordinates:
  * time_rain   (time_rain) datetime64[ns] 1993-01-01T06:00:00 ... 2018-12-31...
  * time_swrad  (time_swrad) datetime64[ns] 1993-01-01T12:00:00 ... 2018-12-3...
  * lon         (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.5 13.75 14.0
  * lat         (lat) float32 32.0 32.25 32.5 32.75 ... 65.25 65.5 65.75 66.0
Data variables:
    rain        (time_rain, lat, lon) float32 ...
    swrad       (time_swrad, lat, lon) float32 ...

In [16]:
data_fc.rain

<xarray.DataArray 'rain' (time_rain: 18992, lat: 137, lon: 153)>
[398091312 values with dtype=float32]
Coordinates:
  * time_rain  (time_rain) datetime64[ns] 1993-01-01T06:00:00 ... 2018-12-31T...
  * lon        (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.25 13.5 13.75 14.0
  * lat        (lat) float32 32.0 32.25 32.5 32.75 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          kilogram meter-2 second-1
    long_name:      rain fall rate
    standard_name:  precipitation_flux
    time:           time_rain

In [17]:
data_fc.swrad

<xarray.DataArray 'swrad' (time_swrad: 9496, lat: 137, lon: 153)>
[199045656 values with dtype=float32]
Coordinates:
  * time_swrad  (time_swrad) datetime64[ns] 1993-01-01T12:00:00 ... 2018-12-3...
  * lon         (lon) float32 -24.0 -23.75 -23.5 -23.25 ... 13.5 13.75 14.0
  * lat         (lat) float32 32.0 32.25 32.5 32.75 ... 65.25 65.5 65.75 66.0
Attributes:
    units:          watt meter-2
    long_name:      solar shortwave radiation flux
    standard_name:  net_downward_shortwave_flux_at_sea_water_surface
    time:           time_swrad